In [115]:
cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [116]:
ls

4_autoML_telescope_TPOT.ipynb  keras_sub4.csv    pytorch_sub2.csv
keras_sub1000.csv              keras_sub5.csv    sample_submission.csv
keras_sub100.csv               keras_sub6.csv    Sunkeras.ipynb
keras_sub1.csv                 keras_sub7.csv    test_df.csv
keras_sub2.csv                 keras_sub.csv     torch_sub.csv
keras_sub300.csv               Pytorch.ipynb     train_df.csv
keras_sub3.csv                 pytorch_sub1.csv


In [0]:
import numpy as np
import pandas as pd

In [0]:
train = pd.read_csv("train_df.csv")
test = pd.read_csv("test_df.csv")
sample_submission = pd.read_csv("sample_submission.csv")

In [0]:
X = train.iloc[:, 1:].values / 255
y = train.iloc[:, 0].values
test = test.iloc[:, 1:].values / 255

Dataset 클래스
   - pytorch의 Dataset 클래스를 상속받아 각 클래스 생성 가능
   - 기본적으로 3개의 함수를 상속받아 생성

In [120]:
len(X[1])

784

In [0]:
from torch.utils.data import Dataset, DataLoader
import torch
class TrainDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform # 데이터 타입을 Tensor 형태로 변경
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        X, y = self.X[idx], self.y[idx]
        return torch.tensor(X).float(), torch.tensor(y)

class TestDataset(Dataset):
    def __init__(self, X, y=None, transform=None):
        self.X = X
        self.y = y
        self.transform = transform
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        X = self.X[idx]
        return torch.tensor(X).float()

In [0]:
traindataset = TrainDataset(X, y)
testdataset = TestDataset(test)

In [123]:
# tensor 로 바뀐 데이터..?
for i in traindataset:
  print(len(i),i)

  break

2 (tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0706, 0.9961, 0.9804, 1.0000, 0.6000,
        0.0118, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.000

DataLoader
   - dataset : 우리가 불러올 데이터 셋
   - batch_size = batch 단위 만큼 데이터를 뽑아옴
   - shuffle : 데이터를 shuffle할 것인지

In [0]:
trainloader = DataLoader(traindataset, batch_size=256, shuffle=True, num_workers=2)
testloader = DataLoader(testdataset, batch_size=256, shuffle=False, num_workers=2)

mnist image
   - 28 x 28 픽셀로 구성.
   - input으로 들어갈 경우 28 x 28로 변경

기본적인 네트워크 구조 만들기
   - class를 이용해 생성 *nn.Module 상속
      - init() 에서 super를 통해 상속
   - 기본적으로 init, forword 함수 생성
   - init 함수에서 super(classname, self).init() 정의

PyTorch: 사용자 정의 nn 모듈
   - forward에 사용할 layer를 생성
   - 입력 Variable을 받아 다른 모듈 또는 Variable의 autograd 연산을 사용하여 출력 Variable을 생성하는 forward 를 정의

nn vs F
   - nn의 경우 레이어 안에서 weight 공유가 가능
   - F의 경우 단순한 연산 기능만

Pytorch : nn
   - 연산 그래프와 autograd는 복잡한 연산자를 정의하고 도함수(derivative)를 자동으로 계산
   - 모듈은 입력 Variable을 받고 출력 Variable을 계산

In [0]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()

criterion : 손실함수 지정
   - nn.CrossEntropyLoss()의 경우 기본적으로 LogSoftmax()가 내장

optimizer : 최적화 함수 지정
   - model.parameters()를 통해 model의 파라미터들을 할당
   - lr : learning_rate 지정

In [0]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.00001)

 GPU 연산을 하기위해선 텐서를 GPU 연산이 가능한 자료형으로 변환해야함

 GPU 연산이 가능한 Tensor를 만드는 것은 device 인수에 GPU 디바이스 객체를 입력하거나 문자열을 입력

In [127]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
print(device)

net = net.to(device)

cuda


In [128]:
num_epochs = 20
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
    print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 20))
    running_loss = 0.0

print('Finished Training')

[1,   165] loss: 18.599
[2,   165] loss: 17.159
[3,   165] loss: 14.395
[4,   165] loss: 11.057
[5,   165] loss: 8.210
[6,   165] loss: 6.185
[7,   165] loss: 4.824
[8,   165] loss: 3.925
[9,   165] loss: 3.323
[10,   165] loss: 2.881
[11,   165] loss: 2.580
[12,   165] loss: 2.328
[13,   165] loss: 2.132
[14,   165] loss: 1.971
[15,   165] loss: 1.848
[16,   165] loss: 1.740
[17,   165] loss: 1.660
[18,   165] loss: 1.579
[19,   165] loss: 1.523
[20,   165] loss: 1.458
Finished Training


In [129]:
correct = 0
total = 0
preds = []
with torch.no_grad():
    for data in testloader:
        images = data
        images = images.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        for pred in predicted:
          preds.append(pred.cpu().numpy())
print(preds[:10])

[array(8), array(0), array(5), array(3), array(8), array(1), array(9), array(6), array(6), array(0)]


In [0]:
def makeSub(preds, sample, i):
  sample['Category'] = pd.Series(preds)
  sample.to_csv('pytorch_sub{}.csv'.format(i), index = False)

In [0]:
makeSub(preds, sample_submission, 1)

# ANN

In [0]:
# 출처 : https://www.kaggle.com/kanncaa1/pytorch-tutorial-for-deep-learning-lovers
class ANNModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ANNModel, self).__init__()
        # Linear function 1: 784 --> 100
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity 1
        self.relu1 = nn.ReLU()
        # Linear function 2: 100 --> 100
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 2
        self.tanh2 = nn.Tanh()       
        # Linear function 3: 100 --> 100
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 3
        self.elu3 = nn.ELU()        
        # Linear function 4 (readout): 100 --> 10
        self.fc4 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        # Non-linearity 1
        out = self.relu1(out)       
        # Linear function 2
        out = self.fc2(out)
        # Non-linearity 2
        out = self.tanh2(out)       
        # Linear function 2
        out = self.fc3(out)
        # Non-linearity 2
        out = self.elu3(out)      
        # Linear function 4 (readout)
        out = self.fc4(out)
        return out

In [0]:
# instantiate ANN
input_dim = 28*28 # 그림 사이즈..?
hidden_dim = 150 
output_dim = 10

model = ANNModel(input_dim, hidden_dim, output_dim)

In [0]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.1) # 확률적 경사 하강법
optimizer = optim.Adam(model.parameters(), lr=0.00001) # RAdam 을 적용해보고 싶었는데 구글링 실패..

In [143]:
if torch.cuda.is_available():
    device = torch.device("cuda") 
print(device)

model = model.to(device)

cuda


In [144]:
from torch.autograd import Variable
num_epochs = 30
count = 0
loss_list = []
iteration_list = []
for epoch in range(num_epochs):
    for i, data in enumerate(trainloader):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Clear gradients
        optimizer.zero_grad()
        # Forward propagation
        outputs = model(inputs)
        # Calculate softmax and ross entropy loss
        loss = criterion(outputs, labels)
        # Calculating gradients
        loss.backward()      
        # Update parameters
        optimizer.step()
        
        # print statistics
        running_loss += loss.item()
    print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, running_loss / 20))
    running_loss = 0.0

print('Finished Training')

[1,   165] loss: 18.851
[2,   165] loss: 18.315
[3,   165] loss: 17.137
[4,   165] loss: 15.156
[5,   165] loss: 12.800
[6,   165] loss: 10.630
[7,   165] loss: 8.848
[8,   165] loss: 7.430
[9,   165] loss: 6.304
[10,   165] loss: 5.415
[11,   165] loss: 4.709
[12,   165] loss: 4.124
[13,   165] loss: 3.657
[14,   165] loss: 3.286
[15,   165] loss: 2.973
[16,   165] loss: 2.715
[17,   165] loss: 2.517
[18,   165] loss: 2.335
[19,   165] loss: 2.196
[20,   165] loss: 2.073
[21,   165] loss: 1.954
[22,   165] loss: 1.853
[23,   165] loss: 1.781
[24,   165] loss: 1.696
[25,   165] loss: 1.635
[26,   165] loss: 1.577
[27,   165] loss: 1.524
[28,   165] loss: 1.485
[29,   165] loss: 1.437
[30,   165] loss: 1.412
Finished Training


In [145]:
correct = 0
total = 0
preds = []
with torch.no_grad():
    for data in testloader:
        images = data
        images = images.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        for pred in predicted:
          preds.append(pred.cpu().numpy())
print(preds[:10])

[array(8), array(0), array(5), array(3), array(8), array(1), array(9), array(6), array(6), array(0)]


In [0]:
# makeSub(preds, sample_submission, 2)
makeSub(preds, sample_submission, 3)